In [ ]:
import re
import numpy as np

with open('input.txt', 'r') as f:
  lines = f.read().splitlines()

In [ ]:
disk_map = list(map(int, list(lines[0])))

# Files in disk map as tuples (offset, file_id, length).
blocks = []
offset = 0
file_id = 0

for i in range(len(disk_map)):
  if i % 2 == 0:
    blocks.append((offset, file_id, disk_map[i]))
    file_id += 1
  offset += disk_map[i]

def defrag(blocks):
  result = []
  write_offset = 0

  i = 0
  while i < len(blocks):
    (head_offset, head_file_id, head_length) = blocks[i]

    # Find out if there's already a block starting at write_offset, or if we need to fill some free space.
    if head_offset == write_offset:
      result.append(blocks[i])
      write_offset += head_length
      i += 1
    else:
      # Fill free space by moving in blocks (or partial blocks) from the tail end of the disk.
      free_space = head_offset - write_offset
      (tail_offset, tail_file_id, tail_length) = blocks[-1]
      if tail_length <= free_space:
        result.append((write_offset, tail_file_id, tail_length))
        write_offset += tail_length
        blocks = blocks[:-1]
      else:
        result.append((write_offset, tail_file_id, free_space))
        write_offset += free_space
        blocks = blocks[:-1] + [(tail_offset, tail_file_id, tail_length - free_space)]

  return result

blocks_defragged = defrag(blocks)

In [ ]:
def checksum(blocks):
  result = 0
  for (b_offset, b_file_id, b_length) in blocks:
    for i in range(b_length):
      result += (b_offset + i) * b_file_id
  return result

checksum(blocks_defragged)